# Getting started

_pyratings_ is made for working with credit ratings in a professionally and
efficiently manner.

The following code demonstrates how to compute an average rating on a portfolio basis
as well as the weighted average rating factor (WARF). This should give you a thorough
introduction into the usage of _pyratings_.

## Preliminary tasks

As a first step and as a basis for both tasks, we are going to import a portfolio
into a `pd.DataFrame`. We'll call it `port_df`.

In [1]:
import pandas as pd

import pyratings as rtg

port_df = pd.read_excel("portfolio.xlsx")
port_df.head()

,ISIN,weight,BB Comp,SP,Moody,Fitch
0,ISIN00000001,0.518515,AAA,NR,Aaa *-,AAA
1,ISIN00000002,0.950810,AAA,AA+,Aaa,AAA
2,ISIN00000003,0.497176,AA+ *-,AA+,Aa2,NaN
3,ISIN00000004,0.648453,NR,NaN,NR,AA-u
4,ISIN00000005,0.674328,NR,NaN,NR,AA-u


By looking at the very first rows of `port_df`, we can see that some
ratings do have a _watch_ attached. This is marked by the `*-` suffix, which
follows the actual rating after a blank.
Other ratings are unsolicited. In this case, the rating is followed by the letter
`u`.
In order to translate ratings into scores, we first need to clean up the ratings,
that is, we need to strip off _watches_ etc. We are going to use the
[get_pure_ratings](../api_reference/#pyratings.clean.get_pure_ratings) function.
This function works on strings, so we need to make sure to pass the relevant columns.

In [2]:
ratings_clean_df = rtg.get_pure_ratings(
    port_df.loc[:, ["BB Comp", "SP", "Moody", "Fitch"]]
)
ratings_clean_df.head()

,BB Comp_clean,SP_clean,Moody_clean,Fitch_clean
0,AAA,NR,Aaa,AAA
1,AAA,AA+,Aaa,AAA
2,AA+,AA+,Aa2,NaN
3,NR,NaN,NR,AA-
4,NR,NaN,NR,AA-


As you can see, the suffix "\_clean", has automatically been added to the column
headings. Let's add these _clean_ ratings to `port_df` by simple concatenation.

In [3]:
port_df = pd.concat([port_df, ratings_clean_df], axis=1)
port_df.head()

,ISIN,weight,BB Comp,SP,Moody,Fitch,BB Comp_clean,SP_clean,Moody_clean,Fitch_clean
0,ISIN00000001,0.518515,AAA,NR,Aaa *-,AAA,AAA,NR,Aaa,AAA
1,ISIN00000002,0.950810,AAA,AA+,Aaa,AAA,AAA,AA+,Aaa,AAA
2,ISIN00000003,0.497176,AA+ *-,AA+,Aa2,NaN,AA+,AA+,Aa2,NaN
3,ISIN00000004,0.648453,NR,NaN,NR,AA-u,NR,NaN,NR,AA-
4,ISIN00000005,0.674328,NR,NaN,NR,AA-u,NR,NaN,NR,AA-


Now, in order to compute an average rating or WARF, respectively, we need to
consolidate the ratings on a
row basis, i.e. we need to assign one and only one concrete rating to an individual
security.<br>
There are several solutions to this problem. Oftentimes, you may want to compute the
__worst__ rating among all rating agencies. It is also fairly common to calculate the
__second best__ rating.<br>
_pyratings_ offers a number of functions in order to accomplish this task, such as

* [get_best_ratings](../api_reference/#pyratings.consolidate.get_best_ratings)
* [get_second_best_ratings](../api_reference/#pyratings.consolidate.get_second_best_ratings)
* [get_worst_ratings](../api_reference/#pyratings.consolidate.get_worst_ratings)

Let's compute the __worst ratings__ and directly concatenate the newly computed
column to our existing `port_df` in one batch. _pyratings_ automatically names the
new column "worst_rtg".

In [4]:
port_df = pd.concat(
    [
        port_df,
        rtg.get_worst_ratings(
            port_df[["BB Comp_clean", "SP_clean", "Moody_clean", "Fitch_clean"]],
            rating_provider_input=["Bloomberg", "S&P", "Moody's", "Fitch"],
            rating_provider_output="S&P"
        )
    ],
    axis=1,
)

port_df.head()

,ISIN,weight,BB Comp,SP,Moody,Fitch,BB Comp_clean,SP_clean,Moody_clean,Fitch_clean,worst_rtg
0,ISIN00000001,0.518515,AAA,NR,Aaa *-,AAA,AAA,NR,Aaa,AAA,AAA
1,ISIN00000002,0.950810,AAA,AA+,Aaa,AAA,AAA,AA+,Aaa,AAA,AA+
2,ISIN00000003,0.497176,AA+ *-,AA+,Aa2,NaN,AA+,AA+,Aa2,NaN,AA
3,ISIN00000004,0.648453,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-
4,ISIN00000005,0.674328,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-


## Computing the average rating of a portfolio

The column <i>worst_rtg</i> will be used in order to translate the ratings into scores
according to the following table.

| Moody’s |  S&P | Fitch | Bloomberg | DBRS |  ICE | Score |
|:-------:|:----:|:-----:|:---------:|:----:|:----:|------:|
|   Aaa   |  AAA |  AAA  |    AAA    |  AAA |  AAA |     1 |
|   Aa1   |  AA+ |  AA+  |    AA+    |  AAH |  AA+ |     2 |
|   Aa2   |  AA  |   AA  |     AA    |  AA  |  AA  |     3 |
|   Aa3   |  AA- |  AA-  |    AA-    |  AAL |  AA- |     4 |
|    A1   |  A+  |   A+  |     A+    |  AH  |  A+  |     5 |
|    A2   |   A  |   A   |     A     |   A  |   A  |     6 |
|    A3   |  A-  |   A-  |     A-    |  AL  |  A-  |     7 |
|   Baa1  | BBB+ |  BBB+ |    BBB+   | BBBH | BBB+ |     8 |
|   Baa2  |  BBB |  BBB  |    BBB    |  BBB |  BBB |     9 |
|   Baa3  | BBB- |  BBB- |    BBB-   | BBBL | BBB- |    10 |
|   Ba1   |  BB+ |  BB+  |    BB+    |  BBH |  BB+ |    11 |
|   Ba2   |  BB  |   BB  |     BB    |  BB  |  BB  |    12 |
|   Ba3   |  BB- |  BB-  |    BB-    |  BBL |  BB- |    13 |
|    B1   |  B+  |   B+  |     B+    |  BH  |  B+  |    14 |
|    B2   |   B  |   B   |     B     |   B  |   B  |    15 |
|    B3   |  B-  |   B-  |     B-    |  BL  |  B-  |    16 |
|   Caa1  | CCC+ |  CCC+ |    CCC+   | CCCH | CCC+ |    17 |
|   Caa2  |  CCC |  CCC  |    CCC    |  CCC |  CCC |    18 |
|   Caa3  | CCC- |  CCC- |    CCC-   | CCCL | CCC- |    19 |
|    Ca   |  CC  |   CC  |     CC    |  CC  |  CC  |    20 |
|    C    |   C  |   C   |     C     |   C  |   C  |    21 |
|    D    |   D  |   D   |    DDD    |   D  |   D  |    22 |
|    SD   |  SD  |   SD  |    ---    |  --- |  --- |    22 |

The function [get_scores_from_ratings](../api_reference/#pyratings.get_scores.get_scores_from_ratings)
will solve this issue. The function needs a rating provider (here: "S&P") in order to
 select an appropriate translation dictionary.

In [5]:
port_scores_df = pd.concat(
    [
        port_df,
        rtg.get_scores_from_ratings(
            ratings=port_df["worst_rtg"],
            rating_provider="S&P")
    ],
    axis=1
)

port_scores_df

,ISIN,weight,BB Comp,SP,Moody,Fitch,BB Comp_clean,SP_clean,Moody_clean,Fitch_clean,worst_rtg,rtg_score_worst_rtg
0,ISIN00000001,0.518515,AAA,NR,Aaa *-,AAA,AAA,NR,Aaa,AAA,AAA,1.0
1,ISIN00000002,0.950810,AAA,AA+,Aaa,AAA,AAA,AA+,Aaa,AAA,AA+,2.0
2,ISIN00000003,0.497176,AA+ *-,AA+,Aa2,NaN,AA+,AA+,Aa2,NaN,AA,3.0
3,ISIN00000004,0.648453,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-,4.0
4,ISIN00000005,0.674328,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,ISIN00000083,2.321185,NR,NaN,Baa1,NaN,NR,NaN,Baa1,NaN,BBB+,8.0
83,ISIN00000084,1.389043,NR,NaN,Baa1,NaN,NR,NaN,Baa1,NaN,BBB+,8.0
84,ISIN00000085,2.296711,BBB+,NR,Baa1,A-,BBB+,NR,Baa1,A-,BBB+,8.0
85,ISIN00000086,1.015105,AAA,AAA,Aaa,NaN,AAA,AAA,Aaa,NaN,AAA,1.0


Finally, we need to compute a weighted average rating score, which we subsequently
convert back into a human-readable rating. For the former, we use the
[get_weighted_average](../api_reference/#pyratings.aggregate.get_weighted_average)
function and for the latter the
[get_ratings_from_scores](../api_reference/#pyratings.get_ratings.get_ratings_from_scores)
function.

In [6]:
avg_rtg_score = rtg.get_weighted_average(
    data=port_scores_df["rtg_score_worst_rtg"],
    weights=port_scores_df["weight"] / 100,
)
print(f"Average rating score: {avg_rtg_score}")

Average rating score: 4.387791453932187


In [7]:
avg_rtg = rtg.get_ratings_from_scores(
    rating_scores=avg_rtg_score, rating_provider="S&P")
print(f"Average portfolio rating: {avg_rtg}")

Average portfolio rating: AA-


## Computing portfolio WARF

To compute the portfolio's WARF, we are again going to use __worst ratings__.

In [8]:
port_df.head()

,ISIN,weight,BB Comp,SP,Moody,Fitch,BB Comp_clean,SP_clean,Moody_clean,Fitch_clean,worst_rtg
0,ISIN00000001,0.518515,AAA,NR,Aaa *-,AAA,AAA,NR,Aaa,AAA,AAA
1,ISIN00000002,0.950810,AAA,AA+,Aaa,AAA,AAA,AA+,Aaa,AAA,AA+
2,ISIN00000003,0.497176,AA+ *-,AA+,Aa2,NaN,AA+,AA+,Aa2,NaN,AA
3,ISIN00000004,0.648453,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-
4,ISIN00000005,0.674328,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-


The column <i>worst_rtg</i> will be used in order to translate the ratings into WARF
according to the following table.

| Moody’s |  S&P | Fitch |  ICE | DBRS | Bloomberg | Score |  WARF | MinWARF* | MaxWARF* |
|:-------:|:----:|:-----:|:----:|:----:|:---------:|------:|------:|---------:|:--------:|
|   Aaa   |  AAA |  AAA  |  AAA |  AAA |    AAA    |     1 |     1 |        1 |        5 |
|   Aa1   |  AA+ |  AA+  |  AA+ |  AAH |    AA+    |     2 |    10 |        5 |       15 |
|   Aa2   |  AA  |   AA  |  AA  |  AA  |     AA    |     3 |    20 |       15 |       30 |
|   Aa3   |  AA- |  AA-  |  AA- |  AAL |    AA-    |     4 |    40 |       30 |       55 |
|    A1   |  A+  |   A+  |  A+  |  AH  |     A+    |     5 |    70 |       55 |       95 |
|    A2   |   A  |   A   |   A  |   A  |     A     |     6 |   120 |       95 |      150 |
|    A3   |  A-  |   A-  |  A-  |  AL  |     A-    |     7 |   180 |      150 |      220 |
|   Baa1  | BBB+ |  BBB+ | BBB+ | BBBH |    BBB+   |     8 |   260 |      220 |      310 |
|   Baa2  |  BBB |  BBB  |  BBB |  BBB |    BBB    |     9 |   360 |      310 |      485 |
|   Baa3  | BBB- |  BBB- | BBB- | BBBL |    BBB-   |    10 |   610 |      485 |      775 |
|   Ba1   |  BB+ |  BB+  |  BB+ |  BBH |    BB+    |    11 |   940 |      775 |     1145 |
|   Ba2   |  BB  |   BB  |  BB  |  BB  |     BB    |    12 |  1350 |     1145 |     1558 |
|   Ba3   |  BB- |  BB-  |  BB- |  BBL |    BB-    |    13 |  1766 |     1558 |     1993 |
|    B1   |  B+  |   B+  |  B+  |  BH  |     B+    |    14 |  2220 |     1993 |     2470 |
|    B2   |   B  |   B   |   B  |   B  |     B     |    15 |  2720 |     2470 |     3105 |
|    B3   |  B-  |   B-  |  B-  |  BL  |     B-    |    16 |  3490 |     3105 |     4130 |
|   Caa1  | CCC+ |  CCC+ | CCC+ | CCCH |    CCC+   |    17 |  4770 |     4130 |     5635 |
|   Caa2  |  CCC |  CCC  |  CCC |  CCC |    CCC    |    18 |  6500 |     5635 |     7285 |
|   Caa3  | CCC- |  CCC- | CCC- | CCCL |    CCC-   |    19 |  8070 |     7285 |     9034 |
|    Ca   |  CC  |   CC  |  CC  |  CC  |     CC    |    20 |  9998 |     9034 |   9998.5 |
|    C    |   C  |   C   |   C  |   C  |     C     |    21 |  9999 |   9998.5 |   9999.5 |
|    D    |   D  |   D   |   D  |   D  |    DDD    |    22 | 10000 |   9999.5 |    10000 |

`MinWARF` is inclusive, while `MaxWARF` is exclusive.

The function [get_warf_from_ratings](../api_reference/#pyratings.get_warf.get_warf_from_ratings)
will solve this issue.

In [9]:
port_warf_df = pd.concat(
    [
        port_df,
        rtg.get_warf_from_ratings(
            ratings=port_df["worst_rtg"],
            rating_provider="S&P",
        )
    ],
    axis=1
)

port_warf_df

,ISIN,weight,BB Comp,SP,Moody,Fitch,BB Comp_clean,SP_clean,Moody_clean,Fitch_clean,worst_rtg,warf_worst_rtg
0,ISIN00000001,0.518515,AAA,NR,Aaa *-,AAA,AAA,NR,Aaa,AAA,AAA,1.0
1,ISIN00000002,0.950810,AAA,AA+,Aaa,AAA,AAA,AA+,Aaa,AAA,AA+,10.0
2,ISIN00000003,0.497176,AA+ *-,AA+,Aa2,NaN,AA+,AA+,Aa2,NaN,AA,20.0
3,ISIN00000004,0.648453,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-,40.0
4,ISIN00000005,0.674328,NR,NaN,NR,AA-u,NR,NaN,NR,AA-,AA-,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,ISIN00000083,2.321185,NR,NaN,Baa1,NaN,NR,NaN,Baa1,NaN,BBB+,260.0
83,ISIN00000084,1.389043,NR,NaN,Baa1,NaN,NR,NaN,Baa1,NaN,BBB+,260.0
84,ISIN00000085,2.296711,BBB+,NR,Baa1,A-,BBB+,NR,Baa1,A-,BBB+,260.0
85,ISIN00000086,1.015105,AAA,AAA,Aaa,NaN,AAA,AAA,Aaa,NaN,AAA,1.0


Finally, we need to compute the WARF on a portfolio basis, which we subsequently
convert back into a human-readable rating. For the former, we use the
[get_weighted_average](../api_reference/#pyratings.aggregate.get_weighted_average)
function and the
[get_ratings_from_warf](../api_reference/#pyratings.get_ratings.get_ratings_from_warf)
function for the latter.

In [10]:
avg_warf = rtg.get_weighted_average(
    data=port_warf_df["warf_worst_rtg"], weights=port_warf_df["weight"] / 100
)

print(f"WARF: {avg_warf}")

WARF: 165.57723587360007


Let us translate `avg_warf` into a Moody's rating.

In [11]:
avg_warf_equivalent_rating = rtg.get_ratings_from_warf(
    warf=avg_warf, rating_provider="Moody")

print(f"Portfolio equivalent WARF rating: {avg_warf_equivalent_rating}")

Portfolio equivalent WARF rating: A3
